### Featurizing text data with tfidf weighted word -vectors

In [1]:
import pandas as pd
import re
import numpy as np
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import sys
import os
import spacy
from tqdm import tqdm

In [2]:
df=pd.read_csv('train.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404289 non-null  object
 4   question2     404288 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [3]:
df=df.fillna(' ')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 404290 entries, 0 to 404289
Data columns (total 6 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            404290 non-null  int64 
 1   qid1          404290 non-null  int64 
 2   qid2          404290 non-null  int64 
 3   question1     404290 non-null  object
 4   question2     404290 non-null  object
 5   is_duplicate  404290 non-null  int64 
dtypes: int64(4), object(2)
memory usage: 18.5+ MB


In [5]:
df['question1']=df['question1'].apply(lambda x:str(x))
df['question2']=df['question2'].apply(lambda x:str(x))

In [6]:
questions=list(df['question1'])+list(df['question2'])
tfidf=TfidfVectorizer(lowercase=False)
tfidf.fit_transform(questions)

word2tfidf=dict(zip(tfidf.get_feature_names(),tfidf.idf_))

C:\Python310\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [7]:
!python -m spacy download en_core_web_lg

!python -m spacy download en

     -------------------------------------- 400.7/400.7 MB 1.8 MB/s eta 0:00:00
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')


     ---------------------------------------- 12.8/12.8 MB 3.4 MB/s eta 0:00:00
[!] As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the full
pipeline package name 'en_core_web_sm' instead.
[+] Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [8]:
nlp=spacy.load('en_core_web_lg')
x=nlp('man')
len(x.vector)

300

In [ ]:
vecs1=[]
for qu1 in tqdm(list(df['question1'])):
    doc1=nlp(qu1)
    mean_vec1=np.zeros([len(doc1),len(doc1[0].vector)])
    for word1 in doc1:
        vec1=word1.vector
        try:
            idf=word2tfidf[str(word1)]
        except:
            idf=0
        mean_vec1 += vec1 * idf
    mean_vec1=mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
df['q1_feats_m']=vecs1

  7%|█████▏                                                                   | 28782/404290 [08:14<1:17:55, 80.31it/s]

In [ ]:
df['q1_feats_m'].head()

In [ ]:
vecs2=[]
for qu2 in tqdm(list(df['question2'])):
    doc2=nlp(qu2)
    mean_vec2=np.zeros([len(doc2),len(doc2[0].vector)])
    for word2 in doc2:
        vec2=word2.vector
        try:
            idf=word2tfidf[str(word2)]
        except:
            idf=0
        mean_vec2+=vec2*idf
    mean_vec2=mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
df['q2_feats_m']=list(vecs2)



In [ ]:
dfnlp=pd.read_csv('nlp_features.csv')

df2=dfnlp.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)

In [ ]:
df2.head()

In [ ]:
df.head()

In [ ]:
df=df.drop(['qid1','qid2','question1','question2'],axis=1)

In [ ]:
df3_q1=pd.DataFrame(df.q1_feats_m.values.tolist(),index=df.index)
df3_q2=pd.DataFrame(df.q2_feats_m.values.tolist(),index=df.index)

In [ ]:
df3_q1.shape

In [ ]:
df3_q1['id']=df['id']
df3_q2['id']=df['id']

df3_q1.head()


In [ ]:
df1.shape

In [ ]:
df1  = df3_q1.merge(df3_q2, on='id',how='left')

In [ ]:
df1.info()

In [ ]:
df=df2.merge(df1,on='id',how='left')

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
df.to_csv('final3.csv')

In [ ]:
df5=pd.read_csv('final3.csv')

df5.head()